In [1]:
# Collecting tweets from zenodo twitter dataset

# Download tweepy to use Twitter API
!pip3 install tweepy
import tweepy

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


In [ ]:
# Open file with Twitter developer keys
keys_file =open("TwitterAPI_Keys_Secret.txt", "r")
keys = keys_file.readlines()

# Twitter developer keys
consumer_key = str(keys[0])
consumer_secret = str(keys[1])
access_token = str(keys[2])
access_token_secret = str(keys[3])

consumer_key = consumer_key[:len(consumer_key)-1]
consumer_secret = consumer_secret[:len(consumer_secret)-1]
access_token = access_token[:len(access_token)-1]
access_token_secret = access_token_secret[:len(access_token_secret)-1]


In [17]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [19]:
import csv

tsv_file = "full_dataset-clean.tsv"
data = open(tsv_file)
read_tsv = csv.reader(data, delimiter="\t")

row_count = 1
tweet_ids = []
for row in read_tsv:
    if row_count == 1:
        row_count += 1
        continue
    tweet_ids.append(row[0])
    print(row_count, row[0])
    row_count += 1
    if row_count == 20:
        break

2 1212360731695427584
3 1213249613735460865
4 1213330173736738817
5 1214195710301618178
6 1214198629553950721
7 1214917256414662656
8 1214936881563082752
9 1214978895914684418
10 1215028015438192640
11 1215081761249615872
12 1215095787002028033
13 1215104980908236800
14 1215110626411978753
15 1215112006363009024
16 1215114820749594626
17 1215166339352080384
18 1215211709142577152
19 1215221343479726081


In [34]:
#print(tweet_ids)
tweet = api.get_status(tweet_ids[1])
print(tweet.text)

Coronavirus | Human Coronavirus Types | CDC https://t.co/IxoxVRarIb
